#Integracion de Datos

## Preparación del ambiente

In [ ]:
! pip install --upgrade pip

! pip install unidecode

! pip install wget

! pip install ydata-profiling

##! pip install --upgrade Pillow

import pandas as pd
import wget

In [ ]:

wget.download('https://github.com/VictoriaRincon/IntegracionDeDatos/raw/main/Fuentes/Reporte_semanal_xx2023.xlsx')
wget.download('https://github.com/VictoriaRincon/IntegracionDeDatos/raw/main/Fuentes/alimentos.xlsx')
wget.download('https://github.com/VictoriaRincon/IntegracionDeDatos/raw/main/Fuentes/cadena_mes_2023.xlsx')
wget.download('https://github.com/VictoriaRincon/IntegracionDeDatos/raw/main/Fuentes/precios_mes_2023.xlsx')
wget.download('https://github.com/VictoriaRincon/IntegracionDeDatos/raw/main/Fuentes/v_productos_real.xlsx')

In [ ]:
v_productos_real = pd.read_excel("v_productos_real.xlsx")
alimentos = pd.read_excel("alimentos.xlsx")

In [ ]:
# Agregar una columna 'id_a' que contiene el número de la tupla (índice de fila)
alimentos['id_a'] = alimentos.index

##Filtro de v_productos_real en v_productos_real_libre_de_gluten

In [ ]:
filtro = v_productos_real['NOMBRE'].str.contains('GLUTEN|SIN TACC', case=False, na=False)
v_productos_real_libre_de_gluten = v_productos_real[filtro]

In [ ]:
columnas_a_eliminar = ['IDPROD', 'RUBRO', 'NRO_HAB','IDCAPI','IDCLASE','CLASE','VIDA_UTIL','PAIS','DPTO','ENVASE','MATERIAL_ENVASE','PRESENTACION','ESTADO','FDESDE_HAB','FHASTA_HAB','RUC']

# Eliminando las columnas especificadas
v_productos_real_libre_de_gluten = v_productos_real_libre_de_gluten.drop(columns=columnas_a_eliminar)

v_productos_real_libre_de_gluten['prod_real_id'] = v_productos_real_libre_de_gluten.index

##Integración entre v_productos_real_libre_de_gluten y alimentos

In [ ]:
columnas = ['Nombre', 'Marca', 'Categoria','Empresa','Local','Tamaño','prod_real_id','a_id']
productos_sin_gluten = pd.DataFrame(columns=columnas)

for i in alimentos.index:
      mi_array=alimentos.Nombre[i].split(' ')
      for indice in v_productos_real_libre_de_gluten.index:
        if(all(palabra_clave.upper() in v_productos_real_libre_de_gluten.NOMBRE[indice].upper() for palabra_clave in mi_array) and (str(v_productos_real_libre_de_gluten['MARCA'][indice]).lower() == alimentos.Marca[i].lower())):
          tupla=(v_productos_real_libre_de_gluten.NOMBRE[indice],v_productos_real_libre_de_gluten.MARCA[indice],alimentos.Categoría[i],v_productos_real_libre_de_gluten.EMPRESA[indice],v_productos_real_libre_de_gluten.LOCAL[indice],
                  v_productos_real_libre_de_gluten.TAMANIO_PRESENTACION[indice],v_productos_real_libre_de_gluten.prod_real_id[indice],alimentos.id_a[i])
          productos_sin_gluten.loc[len(productos_sin_gluten)] = tupla

In [ ]:
productos_sin_gluten['ps_gluten_id']= productos_sin_gluten.index

##Lectura de 'precios_mes' y 'cadena_mes'

In [ ]:
import chardet

cadena_mes = pd.read_excel("cadena_mes_2023.xlsx", header=1)

# Leer el archivo con la codificación detectada
precios_mes = pd.read_excel('precios_mes_2023.xlsx')

In [ ]:
cadena_mes['c_id']= cadena_mes.index

In [ ]:
precios_mes['p_id'] = precios_mes.index

## Integración 'precios_mes' y 'cadena_mes'

In [ ]:
columnas = ['Producto', 'Marca', 'Especificacion','Precio','Empresa','p_id','c_id']
producto_precio = pd.DataFrame(columns=columnas)
columnas_empresas = ['Devoto','Disco','El Dorado','Macro','Red Market','Ta - Ta','Tienda Inglesa']


for i in precios_mes.index:
    filas_precios_cadena = cadena_mes[(cadena_mes['Presentaciones'].str.contains(precios_mes.producto[i], case=False, na=False))
                                       & (pd.isna(precios_mes.marca[i]) or (not pd.isna(precios_mes.marca[i]) and cadena_mes['Presentaciones'].str.contains(precios_mes.marca[i], case=False, na=False)))
                                       & (cadena_mes['Presentaciones'].str.contains(precios_mes.especificacion[i], case=False,na=False))]
    if (len(filas_precios_cadena) >0):
        for row in filas_precios_cadena.index:
          for nombre in columnas_empresas:
            if(not (cadena_mes[nombre][row]=='--')):
              tupla=(precios_mes.producto[i],precios_mes.marca[i],precios_mes.especificacion[i],cadena_mes[nombre][row],nombre,filas_precios_cadena.c_id[row],precios_mes.p_id[i])
              producto_precio.loc[len(producto_precio)] = tupla

producto_precio['pp_id'] = producto_precio.index

## Integración productos_sin_gluten y producto_precio

In [ ]:
import numpy as np
from unidecode import unidecode


columnas_dataset_final = ['Nombre','Marca','Precio','Categoria','Local','Tamaño','Productores','ps_gluten_id','pp_id','rs_id']
dataset_final = pd.DataFrame(columns=columnas_dataset_final)


filas_precios_frutas_vegetales = producto_precio[pd.isna(producto_precio['Marca'])]
for row in filas_precios_frutas_vegetales.index:
  tupla=(filas_precios_frutas_vegetales.Producto[row],'No Aplica',filas_precios_frutas_vegetales.Precio[row],'FRUTAS-HORTALIZAS Y DERIVADOS',filas_precios_frutas_vegetales.Empresa[row],'1kg.','Mayorista Fruta',-1,filas_precios_frutas_vegetales.pp_id[row],-1)
  dataset_final.loc[len(dataset_final)] = tupla


for i in producto_precio.index:
  producto = producto_precio.Producto[i]
  marca = producto_precio.Marca[i]
  empresa = producto_precio.Empresa[i]


  filas_precios_cadena = productos_sin_gluten[(productos_sin_gluten['Nombre'].str.startswith(unidecode(producto).upper()))]
  if (len (filas_precios_cadena)>0):
    filas_precios_cadena2 = filas_precios_cadena[filas_precios_cadena['Marca'].notna()]
    if (len(filas_precios_cadena2)==len(filas_precios_cadena)):
      filas_precios_cadena_final = filas_precios_cadena2[filas_precios_cadena2['Marca'] == unidecode(marca).upper()]
      if (len(filas_precios_cadena_final)>0):
        for row in filas_precios_cadena_final.index:
          tupla=(filas_precios_cadena_final.Nombre[row],filas_precios_cadena_final.Marca[row],producto_precio.Precio[i],filas_precios_cadena_final.Categoria[row],producto_precio.Empresa[i],producto_precio.Especificacion[i],filas_precios_cadena_final.Empresa[row],filas_precios_cadena_final.ps_gluten_id[row], producto_precio.pp_id[i],-1)
          dataset_final.loc[len(dataset_final)] = tupla

In [ ]:
reporte_semanal = pd.read_excel("Reporte_semanal_xx2023.xlsx",header=1)
reporte_semanal = reporte_semanal.rename(columns={'Unnamed: 0': 'Producto'})
reporte_semanal = reporte_semanal.rename(columns={'$/kg': 'Precio'})
reporte_semanal ['rs_id'] = reporte_semanal.index
reporte_semanal = reporte_semanal.drop(reporte_semanal.index[-1])

In [ ]:
reporte_semanal.info()

In [ ]:
largo =len(dataset_final)
for fila in reporte_semanal.index:
      tupla=(reporte_semanal.Producto[fila],'No Aplica',reporte_semanal.Precio[fila],'FRUTAS-HORTALIZAS Y DERIVADOS','UAM','1kg.','Mayorista Fruta',-1, -1,reporte_semanal.rs_id[fila])
      dataset_final.loc[largo] = tupla
      largo+=1

dataset_final = dataset_final.drop_duplicates(subset=dataset_final.columns.difference(['ps_gluten_id','pp_id']))
dataset_final['eig_id']= dataset_final.index

## Creación Esquema Integrado

In [ ]:
import sqlite3
from datetime import datetime
import pytz

zona_horaria_uruguay = pytz.timezone('America/Montevideo')

conn = sqlite3.connect("base_glutenos.db")
cursor = conn.cursor()

dataset_final.to_sql('esquema_integrado_glutenos', conn, index=False, if_exists='replace')
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect("base_glutenos.db")
cursor = conn.cursor()

cursor.execute('SELECT * FROM esquema_integrado_glutenos')
resultados = cursor.fetchall()
pd.DataFrame(resultados)

## Ejemplos de problemas

In [ ]:
cursor = conn.cursor()

consulta = "SELECT * FROM esquema_integrado_glutenos WHERE LOWER(Nombre) LIKE '%dulce de leche%'"
cursor.execute(consulta)

nombres_columnas = [desc[0] for desc in cursor.description]

resultados = cursor.fetchall()
pd.DataFrame(resultados, columns=nombres_columnas)


In [ ]:
cursor = conn.cursor()

consulta = "SELECT * FROM esquema_integrado_glutenos WHERE Local LIKE '%Ta - Ta%'"
cursor.execute(consulta)

nombres_columnas = [desc[0] for desc in cursor.description]

resultados = cursor.fetchall()
pd.DataFrame(resultados, columns=nombres_columnas)